<a href="https://colab.research.google.com/github/none-304/Coursera_Capstone/blob/main/Finance_Methods_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import packages
import numpy as np

In [2]:
import pandas as pd

In [3]:
from google.colab import files

In [4]:
from google.colab import drive

In [5]:
#Preparing Google Drive
drive.mount('/content/drive')

##### If you are using the attached file, upload the file using the code below #### 
# import files
# from google.colab import files
# files.upload()

##### If you are using python natively, just change the path in the cell below to the path of the file ####

Mounted at /content/drive


In [6]:
#Load up dataset
a=pd.read_sas('/content/drive/MyDrive/w23handout.sas7bdat')


In [7]:
#Checking the columns
a.columns

Index(['GVKEY', 'DATADATE', 'CONM', 'SIC', 'SICH', 'LPERMNO', 'AT', 'SALE'], dtype='object')

In [8]:
### 1st task


#Specifying Assets and Sale parameter
a=a[(a.AT>=5)& (a.SALE>0)]

In [9]:
#importing progress bar
import tqdm

In [10]:
a

,GVKEY,DATADATE,CONM,SIC,SICH,LPERMNO,AT,SALE
0,b'001001',1983-12-31,b'A & M FOOD SERVICES INC',b'5812',NaN,10015.0,14.080,25.395
1,b'001001',1984-12-31,b'A & M FOOD SERVICES INC',b'5812',NaN,10015.0,16.267,32.007
2,b'001001',1985-12-31,b'A & M FOOD SERVICES INC',b'5812',NaN,10015.0,39.495,53.798
3,b'001003',1983-12-31,b'A.A. IMPORTING CO INC',b'5712',NaN,10031.0,8.529,13.793
4,b'001003',1984-12-31,b'A.A. IMPORTING CO INC',b'5712',NaN,10031.0,8.241,13.829
...,...,...,...,...,...,...,...,...
272613,b'345980',2021-12-31,b'CONTEXTLOGIC INC',b'5961',5961.0,20333.0,1283.000,2085.000
272614,b'347007',2021-12-31,b'IMMUNITYBIO INC',b'2836',2836.0,15533.0,468.910,0.934
272615,b'347085',2022-02-28,b'KAROOOOO LTD',b'7370',7370.0,21069.0,200.247,177.987
272617,b'349530',2022-02-28,b'NEXTPLAY TECHNOLOGIES INC',b'7310',7310.0,17324.0,99.753,8.203


In [11]:
#Prepare blank dataframe
c=pd.DataFrame()

In [12]:
a['SALE']

0           25.395
1           32.007
2           53.798
3           13.793
4           13.829
            ...   
272613    2085.000
272614       0.934
272615     177.987
272617       8.203
272619     512.094
Name: SALE, Length: 233663, dtype: float64

In [13]:
a['SALE'].shift(+1)

0              NaN
1           25.395
2           32.007
3           53.798
4           13.793
            ...   
272613    2541.000
272614    2085.000
272615       0.934
272617     177.987
272619       8.203
Name: SALE, Length: 233663, dtype: float64

In [14]:
#Detaching each data based on firm code to create lagged variable & appending it back into one
for b in tqdm.tqdm(set(a.GVKEY)):
  z=a[a.GVKEY==b]
  z['LAGSALE']=z['SALE'].shift(+1)
  z['GSALE']=(z.SALE-z.LAGSALE)/z.LAGSALE
  z['LAGASSETS']=z['AT'].shift(+1)
  z['GASSETS']=(z.AT-z.LAGASSETS)/z.LAGASSETS
  c=c.append(z)



  0%|          | 0/22517 [00:00<?, ?it/s]<ipython-input-14-ffbf2e97b60b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['LAGSALE']=z['SALE'].shift(+1)
<ipython-input-14-ffbf2e97b60b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['GSALE']=(z.SALE-z.LAGSALE)/z.LAGSALE
<ipython-input-14-ffbf2e97b60b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [15]:
#Dropping data points that have null values in the growth variables
c.replace([np.inf,np.nan],inplace=True)
c.dropna(subset=['GSALE','GASSETS'],inplace=True)

In [16]:
#Specifying datadate
c=c[c.DATADATE.dt.year.between(1985,2021)]

In [17]:
#Retain firms that have 3 or more data points
c=c.groupby('GVKEY').filter(lambda x : len(x)>=3)

In [18]:
c

,GVKEY,DATADATE,CONM,SIC,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS
206176,b'061394',2000-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,98.087,7.062,7.184,-0.016982,96.875,0.012511
206177,b'061394',2001-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,97.290,7.012,7.062,-0.007080,98.087,-0.008125
206178,b'061394',2002-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,110.002,6.482,7.012,-0.075585,97.290,0.130661
206179,b'061394',2003-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,111.701,6.415,6.482,-0.010336,110.002,0.015445
262575,b'178610',2007-12-31,b'DUPONT FABROS TECHNOLOGY INC',b'6798',6512.0,92331.0,1454.155,61.274,6.482,-0.010336,110.002,0.015445
...,...,...,...,...,...,...,...,...,...,...,...,...
146537,b'017640',2005-12-31,b'SOUTH FINANCIAL GROUP INC',b'6020',6020.0,10825.0,14319.285,809.507,664.759,0.217745,13798.205,0.037764
146538,b'017640',2006-12-31,b'SOUTH FINANCIAL GROUP INC',b'6020',6020.0,10825.0,14210.516,994.882,809.507,0.228997,14319.285,-0.007596
146539,b'017640',2007-12-31,b'SOUTH FINANCIAL GROUP INC',b'6020',6020.0,10825.0,13877.584,993.667,994.882,-0.001221,14210.516,-0.023429
146540,b'017640',2008-12-31,b'SOUTH FINANCIAL GROUP INC',b'6020',6020.0,10825.0,13602.326,843.724,993.667,-0.150899,13877.584,-0.019835


In [19]:
c.groupby(c.DATADATE.dt.year).agg(['mean','count']).loc[:,['GASSETS','GSALE']]

GASSETS            GSALE      
              mean count       mean count
DATADATE                                 
1985      0.161270  3835   0.239423  3835
1986      0.187807  4300   0.432076  4300
1987      0.196432  4805   0.711318  4805
1988      0.144703  4769   0.670067  4769
1989      0.135888  4683   1.269699  4683
1990      0.112672  4682   0.389779  4682
1991      0.082583  4839   0.189352  4839
1992      0.116529  5160   0.454452  5160
1993      0.174573  6118   0.443134  6118
1994      0.172801  6651   1.529623  6651
1995      0.200260  6958   0.509975  6958
1996      0.233545  7442   0.514548  7442
1997      0.237724  7520   0.705595  7520
1998      0.268382  7362   0.470445  7362
1999      0.227292  7142   0.485699  7142
2000      0.331406  6918   0.737247  6918
2001      0.083779  6436   1.931634  6436
2002      0.047370  6064   0.108612  6064
2003      0.119813  5769   0.325380  5769
2004      0.195485  5716   0.562963  5716
2005      0.157394  5609   1.112172  5609
2006      0.185981  5478   0.471112  5478
2007      0.180010  5288   1.584822  5288
2008      0.086434  5020  43.891065  5020
2009      0.065838  4825   0.209547  4825
2010      0.129043  4733   2.262228  4733
2011      0.143761  4626   0.310483  4626
2012      0.138231  4573   0.317556  4573
2013      0.124936  4629   0.215606  4629
2014      0.136296  4722   0.840444  4722
2015      0.105950  4672   0.581224  4672
2016      0.085717  4568   2.088080  4568
2017      0.128731  4512   0.647299  4512
2018      0.123287  4493   3.337289  4493
2019      0.137319  4458   0.483334  4458
2020      0.230924  4252   1.471923  4252
2021      0.239670  4043   1.743908  4043

In [20]:
##### 2nd task


#Loading up SIC to fama french key database
sic_ff12=pd.read_csv('/content/drive/MyDrive/SIC_to_Fama_French_industry.csv')

In [21]:
#Merging the dataset into one
finaldf=pd.merge(c,sic_ff12,left_on=c.SIC.astype(int),right_on='SIC0')
finaldf

,GVKEY,DATADATE,CONM,SIC_x,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS,SIC_y,SIC0,FF_12,FF_48
0,b'061394',2000-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,98.087,7.062,7.184,-0.016982,96.875,0.012511,6020,6020,11,44
1,b'061394',2001-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,97.290,7.012,7.062,-0.007080,98.087,-0.008125,6020,6020,11,44
2,b'061394',2002-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,110.002,6.482,7.012,-0.075585,97.290,0.130661,6020,6020,11,44
3,b'061394',2003-06-30,b'SOUTHERN BANC CO INC',b'6020',NaN,82305.0,111.701,6.415,6.482,-0.010336,110.002,0.015445,6020,6020,11,44
4,b'009911',1985-12-31,b'C & S SOVRAN CORP',b'6020',4922.0,71889.0,9671.672,1037.447,911.454,0.138233,8286.480,0.167163,6020,6020,11,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197665,b'025921',2004-12-31,b'RAILAMERICA INC',b'4013',4013.0,78065.0,1016.143,395.564,358.381,0.103753,1232.490,-0.175537,4013,4013,12,40
197666,b'025921',2005-12-31,b'RAILAMERICA INC',b'4013',4013.0,78065.0,1147.376,423.682,395.564,0.071083,1016.143,0.129148,4013,4013,12,40
197667,b'025921',2009-12-31,b'RAILAMERICA INC',b'4013',4013.0,93068.0,1598.629,410.603,423.682,-0.030870,1147.376,0.393291,4013,4013,12,40
197668,b'025921',2010-12-31,b'RAILAMERICA INC',b'4013',4013.0,93068.0,1603.653,490.291,410.603,0.194076,1598.629,0.003143,4013,4013,12,40


In [22]:
#removing the fama french category 8, 11, & 12
finaldf=finaldf[(finaldf.FF_12!=8)& (finaldf.FF_12!=11) & (finaldf.FF_12!=12)]

In [23]:
finaldf

,GVKEY,DATADATE,CONM,SIC_x,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS,SIC_y,SIC0,FF_12,FF_48
21847,b'004936',1985-06-30,b'G&K SERVICES INC -CL A',b'7200',6411.0,37955.0,42.852,63.830,56.062,0.138561,37.428,0.144918,7200,7200,9,33
21848,b'004936',1986-06-30,b'G&K SERVICES INC -CL A',b'7200',6411.0,37955.0,52.237,71.586,63.830,0.121510,42.852,0.219010,7200,7200,9,33
21849,b'004936',1987-06-30,b'G&K SERVICES INC -CL A',b'7200',7200.0,37955.0,57.131,80.755,71.586,0.128084,52.237,0.093688,7200,7200,9,33
21850,b'004936',1988-06-30,b'G&K SERVICES INC -CL A',b'7200',7200.0,37955.0,75.876,94.007,80.755,0.164101,57.131,0.328106,7200,7200,9,33
21851,b'004936',1989-06-30,b'G&K SERVICES INC -CL A',b'7200',7200.0,37955.0,83.383,108.817,94.007,0.157541,75.876,0.098938,7200,7200,9,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197648,b'211531',2002-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,13.340,6.174,5.693,0.084490,10.830,0.231764,900,900,1,48
197649,b'211531',2003-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,16.688,6.706,6.174,0.086168,13.340,0.250975,900,900,1,48
197650,b'211531',2004-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,16.536,4.545,6.706,-0.322249,16.688,-0.009108,900,900,1,48
197651,b'211531',2005-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,17.995,7.084,4.545,0.558636,16.536,0.088232,900,900,1,48


In [24]:
#grouping by year
finaldf.groupby(finaldf.DATADATE.dt.year).agg(['mean','count']).loc[:,['GASSETS','GSALE']]

GASSETS           GSALE      
              mean count      mean count
DATADATE                                
1985      0.146037  2598  0.255613  2598
1986      0.181349  2868  0.428591  2868
1987      0.189249  3175  0.456164  3175
1988      0.130056  3124  0.477540  3124
1989      0.130020  3074  0.412672  3074
1990      0.119841  3067  0.384636  3067
1991      0.097398  3199  0.229049  3199
1992      0.122452  3466  0.588060  3466
1993      0.154642  3839  0.467289  3839
1994      0.168064  4108  1.892731  4108
1995      0.220968  4316  0.658752  4316
1996      0.239151  4723  0.492646  4723
1997      0.211249  4821  0.863652  4821
1998      0.216792  4694  0.531797  4694
1999      0.256173  4543  0.464749  4543
2000      0.381179  4458  0.805824  4458
2001      0.072368  4138  0.623567  4138
2002      0.020793  3871  0.136849  3871
2003      0.103248  3627  0.364370  3627
2004      0.200439  3577  0.579264  3577
2005      0.147509  3492  0.533776  3492
2006      0.189821  3414  0.287028  3414
2007      0.181638  3291  2.238621  3291
2008      0.048279  3110  0.537915  3110
2009      0.057812  2958  0.245556  2958
2010      0.151568  2886  3.489090  2886
2011      0.168726  2818  0.402729  2818
2012      0.117493  2768  0.350074  2768
2013      0.123781  2796  0.258714  2796
2014      0.144239  2834  1.221485  2834
2015      0.100416  2805  0.733483  2805
2016      0.081816  2751  3.391053  2751
2017      0.134205  2709  0.979185  2709
2018      0.134130  2715  5.347143  2715
2019      0.155482  2705  0.662877  2705
2020      0.284496  2572  2.434595  2572
2021      0.292165  2454  2.679401  2454